## True Random dice!

This is a notebook to make true random dice out of qubits.
Eventually, this should be able to parse various input strings:
e.g. "3d4" is calculated by creating a 4-state (2 qubit) circuit and running it three times.
For dice sizes that aren't created easily with h gates, make the smallest quantum circuit needed and reroll anything too high.

In [1]:
#required input just for rolls
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile
from qiskit_aer import AerSimulator
import numpy as np

In [6]:
def handle_input(strg):
    #this func will take an input of the form 3d6p2
    #then split and return calls = 3, size = 6, add = 2
    split = strg.split("d")
    if "p" in split[1]:
        split2 = split[1].split("p")
        return(int(split[0]),int(split2[0]),int(split2[1]))
    else:
        #no add
        return(int(split[0]),int(split[1]),0)


def create_circuit(size):
    #take size as int, build a circuit with equal chances of each state
    qubits = int(np.ceil(np.log2(size)))
    qr = QuantumRegister(qubits,'x')
    cr = ClassicalRegister(qubits,'c')
    qc = QuantumCircuit(qr,cr)       
    for i in range (qubits):
        qc.h(i)        
    qc.measure(qr,cr)
    return qc


def count_to_dec(count):
    #takes result.get_counts() and converts to a decimal result
    strg = str(count)
    split = strg.split("'")
    num = int(split[1],2)
    return (num+1)
    
    
    
def main(inp):
    #take in the roll text and run everything!
    #for external usage, outputs all prints in a list
    try:
        calls,size,add = handle_input(inp)
    except:
        print("bad input. Plese provide roll in form of 1d20 or 3d6p2.")
        return ["invalid input"]
    
    if size < 2:
        print ("That's not a real dice!!!")
        return ["invalid input"]

    out = []
    qc = create_circuit(size)
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    total_roll = add
    
    for _ in range (calls):
        reroll = True
        while reroll:
            #sym and run the circuit
            job = simulator.run(compiled_circuit, shots=1)
            result = job.result()
            roll = count_to_dec(result.get_counts())
            #reroll if result is too big
            if roll <= size:
                reroll = False
            else:
                continue

            #show each roll individually
            if calls > 1:
                msg = "d"+str(size)+": "+str(roll)
                print (msg)
                out.append(msg)
        total_roll+=roll
    msg = "you rolled "+str(inp)+" and got "+str(total_roll)+"!"
    print (msg)
    out.append(msg)
    #not necessary in ,ipynb setting
    #return out

In [7]:
#main
main('1d20')

you rolled 1d20 and got 2!
